# Single Agent 

## Setup 
Install Python, SUMO, and required packages.

In [ ]:
# Setup SUMO=1.10.0
!apt-get update --fix-missing
!apt-get install -y libspatialindex-dev xorg
%pip install eclipse-sumo==1.10.0
%env SUMO_HOME=/usr/local/bin/sumo

# Install SMARTS
!git clone https://github.com/huawei-noah/SMARTS
%cd /content/SMARTS
!git checkout develop
%pip install -e .[camera-obs]

# Build scenarios
%cd /content/SMARTS
!scl scenario build --clean scenarios/loop

## Run the single agent example 

In [ ]:
from argument_parser import default_argument_parser

parser = default_argument_parser("single-agent-example")
args = parser.parse_args()

main(
    scenarios=args.scenarios,
    sim_name=args.sim_name,
    headless=args.headless,
    num_episodes=args.episodes,
    seed=args.seed,
)